The three JSON files from data scrape.ipynb are imported to create the cumulative and the mobile JSON files

In [1]:
import pandas as pd
import json
from json import loads, dumps

df_desktop = pd.read_json('inter_desktop_start201507-end202310.json')['articles']
df_desktop = pd.json_normalize(df_desktop)

df_mobile_app = pd.read_json('inter_mobile_app_start201507-end202310.json')['articles']
df_mobile_app = pd.json_normalize(df_mobile_app)

df_mobile_web = pd.read_json('inter_mobile_web_start201507-end202310.json')['articles']
df_mobile_web = pd.json_normalize(df_mobile_web)

In [2]:
df_desktop['month'] = df_desktop['timestamp'].str.slice(start=4,stop=6)
df_desktop['year'] =df_desktop['timestamp'].str.slice(stop=4)
df_desktop['timestamp'] = df_desktop[['year', 'month']].agg('-'.join, axis=1)
df_desktop = df_desktop.drop(columns=['project', 'granularity', 'access', 'agent', 'year', 'month'])
df_desktop['timestamp'] = pd.to_datetime(df_desktop['timestamp'])

In [3]:
df_mobile_app['month'] = df_mobile_app['timestamp'].str.slice(start=4,stop=6)
df_mobile_app['year'] =df_mobile_app['timestamp'].str.slice(stop=4)
df_mobile_app['timestamp'] = df_mobile_app[['year', 'month']].agg('-'.join, axis=1)
df_mobile_app = df_mobile_app.drop(columns=['project', 'granularity', 'access', 'agent', 'year', 'month'])
df_mobile_app['timestamp'] = pd.to_datetime(df_mobile_app['timestamp'])

In [4]:
df_mobile_web['month'] = df_mobile_web['timestamp'].str.slice(start=4,stop=6)
df_mobile_web['year'] =df_mobile_web['timestamp'].str.slice(stop=4)
df_mobile_web['timestamp'] = df_mobile_web[['year', 'month']].agg('-'.join, axis=1)
df_mobile_web = df_mobile_web.drop(columns=['project', 'granularity', 'access', 'agent', 'year', 'month'])
df_mobile_web['timestamp'] = pd.to_datetime(df_mobile_web['timestamp'])

From the below, we have the basic information of the number of views from different platforms.

In [5]:
df_desktop.shape

(128860, 3)

In [6]:
df_desktop.head()

,article,timestamp,views
0,Everything_Everywhere_All_at_Once,2020-01-01,1209
1,Everything_Everywhere_All_at_Once,2020-02-01,2944
2,Everything_Everywhere_All_at_Once,2020-03-01,2612
3,Everything_Everywhere_All_at_Once,2020-04-01,4530
4,Everything_Everywhere_All_at_Once,2020-05-01,3952


In [7]:
df_desktop.describe()

,views
count,1.288600e+05
mean,1.144953e+04
std,3.720301e+04
min,0.000000e+00
25%,5.370000e+02
50%,2.787000e+03
75%,1.188100e+04
max,3.355060e+06


In [8]:
df_mobile_app.shape

(128860, 3)

In [9]:
df_mobile_app.head()

,article,timestamp,views
0,Everything_Everywhere_All_at_Once,2020-01-01,65
1,Everything_Everywhere_All_at_Once,2020-02-01,152
2,Everything_Everywhere_All_at_Once,2020-03-01,120
3,Everything_Everywhere_All_at_Once,2020-04-01,284
4,Everything_Everywhere_All_at_Once,2020-05-01,231


In [10]:
df_mobile_app.describe()

,views
count,128860.000000
mean,875.145887
std,4109.631416
min,0.000000
25%,16.000000
50%,144.000000
75%,824.000000
max,516287.000000


In [12]:
df_mobile_web.shape

(128860, 3)

In [13]:
df_mobile_web.head()

,article,timestamp,views
0,Everything_Everywhere_All_at_Once,2020-01-01,2241
1,Everything_Everywhere_All_at_Once,2020-02-01,4955
2,Everything_Everywhere_All_at_Once,2020-03-01,4427
3,Everything_Everywhere_All_at_Once,2020-04-01,9540
4,Everything_Everywhere_All_at_Once,2020-05-01,7878


In [14]:
df_mobile_web.describe()

,views
count,1.288600e+05
mean,1.822776e+04
std,7.476695e+04
min,0.000000e+00
25%,4.120000e+02
50%,3.001500e+03
75%,1.707425e+04
max,7.736645e+06


### For the JSON file that's specifically for mobile platform, we join on article and timestamp to prepare for the summation of total view from both mobile-web and -app

In [15]:
# join on article and timestamp to prepare for the summation of total view from both mobile-web and -app
# further join on project, granularity and agent to remove redundancy before hand

mobile = pd.merge(df_mobile_app, df_mobile_web, on=['article','timestamp'])
mobile

,article,timestamp,views_x,views_y
0,Everything_Everywhere_All_at_Once,2020-01-01,65,2241
1,Everything_Everywhere_All_at_Once,2020-02-01,152,4955
2,Everything_Everywhere_All_at_Once,2020-03-01,120,4427
3,Everything_Everywhere_All_at_Once,2020-04-01,284,9540
4,Everything_Everywhere_All_at_Once,2020-05-01,231,7878
...,...,...,...,...
128855,Zorba_the_Greek_(film),2023-05-01,493,8134
128856,Zorba_the_Greek_(film),2023-06-01,484,8282
128857,Zorba_the_Greek_(film),2023-07-01,765,14055
128858,Zorba_the_Greek_(film),2023-08-01,1203,12530


In [16]:
mobile['views'] = mobile['views_x']+mobile['views_y']

In [17]:
# remove the redundant columns resulting from joining after the computation

mobile = mobile.drop(columns=['views_x', 'views_y'])
mobile

,article,timestamp,views
0,Everything_Everywhere_All_at_Once,2020-01-01,2306
1,Everything_Everywhere_All_at_Once,2020-02-01,5107
2,Everything_Everywhere_All_at_Once,2020-03-01,4547
3,Everything_Everywhere_All_at_Once,2020-04-01,9824
4,Everything_Everywhere_All_at_Once,2020-05-01,8109
...,...,...,...
128855,Zorba_the_Greek_(film),2023-05-01,8627
128856,Zorba_the_Greek_(film),2023-06-01,8766
128857,Zorba_the_Greek_(film),2023-07-01,14820
128858,Zorba_the_Greek_(film),2023-08-01,13733


### For the JSON file that's for all platforms, we join on article and timestamp to prepare for the summation of total view from both desktop and the resulting DataFrame that's solely about mobile from the above

In [18]:
# join on article and timestamp to prepare for the summation of total view from both mobile and desktop
# further join on project, granularity and agent to remove redundancy before hand

cumulative = pd.merge(mobile, df_desktop, on=['article','timestamp'])
cumulative

,article,timestamp,views_x,views_y
0,Everything_Everywhere_All_at_Once,2020-01-01,2306,1209
1,Everything_Everywhere_All_at_Once,2020-02-01,5107,2944
2,Everything_Everywhere_All_at_Once,2020-03-01,4547,2612
3,Everything_Everywhere_All_at_Once,2020-04-01,9824,4530
4,Everything_Everywhere_All_at_Once,2020-05-01,8109,3952
...,...,...,...,...
128855,Zorba_the_Greek_(film),2023-05-01,8627,4398
128856,Zorba_the_Greek_(film),2023-06-01,8766,3865
128857,Zorba_the_Greek_(film),2023-07-01,14820,5919
128858,Zorba_the_Greek_(film),2023-08-01,13733,5789


In [19]:
cumulative['views'] = cumulative['views_x']+cumulative['views_y']

In [20]:
# remove the redundant columns resulting from joining after the computation

cumulative = cumulative.drop(columns=['views_x', 'views_y'])
cumulative

,article,timestamp,views
0,Everything_Everywhere_All_at_Once,2020-01-01,3515
1,Everything_Everywhere_All_at_Once,2020-02-01,8051
2,Everything_Everywhere_All_at_Once,2020-03-01,7159
3,Everything_Everywhere_All_at_Once,2020-04-01,14354
4,Everything_Everywhere_All_at_Once,2020-05-01,12061
...,...,...,...
128855,Zorba_the_Greek_(film),2023-05-01,13025
128856,Zorba_the_Greek_(film),2023-06-01,12631
128857,Zorba_the_Greek_(film),2023-07-01,20739
128858,Zorba_the_Greek_(film),2023-08-01,19522


In [21]:
# produce the combination of mobile data from mobile-app and -web

out_mobile = mobile.to_json(orient="records")
parsed_mobile = loads(out_mobile)

with open("academy_monthly_mobile_start201507-end202310.json", "a") as outfile:
    outfile.write(json.dumps(parsed_mobile,indent=4))

In [22]:
# produce the combination of data from the resulting mobile DataFrame obtained from the previous codes 
# and the Dataframe of the desktop access

out_cumulative = cumulative.to_json(orient="records")
parsed_cumulative = loads(out_cumulative)

with open("academy_monthly_cumulative_start201507-end202310.json", "a") as outfile:
    outfile.write(json.dumps(parsed_cumulative,indent=4))

In [23]:
# produce the combination of data from the resulting mobile DataFrame obtained from the previous codes 
# and the Dataframe of the desktop access

out_desktop = df_desktop.to_json(orient="records")
parsed_desktop = loads(out_desktop)

with open("academy_monthly_desktop_start201507-end202310.json", "a") as outfile:
    outfile.write(json.dumps(parsed_desktop,indent=4))